# Cloudera AI Inference Workshop

# Model API Integration with Cloudera AI

This notebook demonstrates the flexibility of Cloudera AI inferencing services by showing different ways to interact with deployed models. We'll progress from basic API usage to more complex implementations, showing how easy it is to switch between different models and frameworks.

## Requirements
- Python 3.10 or later
- Access to Cloudera AI console
- Two deployed models: test-model-llama-8b-v2 and deepseek-r1-distill-llama-8b

## Model Setup

Before proceeding, you'll need to gather information from your deployed models in the Cloudera AI console:

1. Go to Cloudera AI console > Model Endpoints
2. Find the models: 
   - Llama 8b
   - Llama 33 70b
3. For each model:
   - Copy the endpoint URL (remove everything after /v1) for example :
   - `https://ai-inference.ainf-cdp.vayb-xokg.cloudera.site/...../modelxyz/openai/v1/chat/completions`
   - would be converted to :
   - `https://ai-inference.ainf-cdp.vayb-xokg.cloudera.site/...../modelxyz/openai/v1`
   - Copy the Model ID
4. Authentication JWT token will be passed to a session, so authentication can happen programatically. For todays lab we'll be copying the token from CAII. See lab docs for details.
   
The first model's information will go into `base_url` and `model_name` variables. The 2nd model will be `llama_url` and `llama_model_name` variables

In [ ]:
from openai import OpenAI
import os
import httpx
import json
from typing import List, Dict, Generator
import asyncio
import time
import re

#### Model enpoint collection
1. Collect Llama 3.1 model endpont details
2. Collect Llama 3.3 model enpoint details

go to cloudera AI and get the following parameters.Cut off tail end of url after '/v1'

**Llama 8b**

In [ ]:
base_url = "https://ml-2dad9e26-62f.go01-dem.ylcu-atmi.cloudera.site/namespaces/serving-default/endpoints/goes---llama3-8b-throughput/v1"
model_name = "meta/llama-3.1-8b-instruct"

**LLama 70b**

In [ ]:
llama_url ="https://ml-2dad9e26-62f.go01-dem.ylcu-atmi.cloudera.site/namespaces/serving-default/endpoints/agentstudiohol/v1"
llama_model_name = "meta/llama-3.3-70b-instruct"

#### Auth setup

Here is the auth token you'll use to connect to your model

In [ ]:
try:
    json.load(open("/tmp/jwt"))["access_token"]
except:
    print('please enter token manually')

In [ ]:
# Load API key
OPENAI_API_KEY = 'eyJraWQiOiIzYzhlNzA3OTEyZmI0NTA1ODE3NzE3YzMyOTU4MmQwMTFjYjlmNTAwIiwidHlwIjoiSldUIiwiYWxnIjoiUlMyNTYifQ.eyJzdWIiOiJvemFyYXRlIiwiYXVkIjoiaHR0cHM6Ly9kZS55bGN1LWF0bWkuY2xvdWRlcmEuc2l0ZSIsImlzcyI6Imh0dHBzOi8vY29uc29sZWF1dGguY2RwLmNsb3VkZXJhLmNvbS84YTFlMTVjZC0wNGMyLTQ4YWEtOGYzNS1iNGE4YzExOTk3ZDMiLCJncm91cHMiOiJjZHBfZGVtb3Nfd29ya2Vyc193dyBjZHBfZGVtby1hd3MtcHJpbSBfY19kZl9kZXZlbG9wXzkxMTQ2M2MgX2NfbWxfYWRtaW5zXzViNTQ3ZDI2IF9jX21sX2J1c2luZXNzX3VzZXJzXzc3ODkxZjJlIF9jX2RmX3ZpZXdfNmY1OWU5ZjMgX2NfZGZfYWRtaW5pc3Rlcl85MTE0NjNjIF9jX21sX3VzZXJzXzc3ODkxZjJlIF9jX2RmX3ZpZXdfOTExNDYzYyBfY19tbF9idXNpbmVzc191c2Vyc182ZWUwZGI5MSBfY19kZl9wdWJsaXNoXzkxMTQ2M2MgX2NfZGZfdmlld185MTE0NjNjMCBfY19lbnZfYXNzaWduZWVzXzkxMTQ2M2MgX2NfcmFuZ2VyX2FkbWluc185MDZiMGJhIF9jX21sX3VzZXJzXzZmNTllOWYzIF9jX21sX3VzZXJzXzRkODNhZDdmIF9jX2Vudl9hc3NpZ25lZXNfOTA2YjBiYSBfY19kZl9kZXZlbG9wXzZmNTllOWYzIF9jX2RmX2FkbWluaXN0ZXJfNmY1OWU5ZjMgX2NfZGZfcHVibGlzaF82ZjU5ZTlmMyBfY19tbF91c2Vyc182ZWUwZGI5MSBfY19tbF9idXNpbmVzc191c2Vyc182ZjU5ZTlmMyBfY19tbF9idXNpbmVzc191c2Vyc185MTE0NjNjIF9jX21sX2FkbWluc183Nzg5MWYyZSBfY19lbnZfYXNzaWduZWVzXzZmNTllOWYzIF9jX3Jhbmdlcl9hZG1pbnNfNmY1OWU5ZjMgX2NfcmFuZ2VyX2FkbWluc185MTE0NjNjIF9jX2RlX3VzZXJzXzkxMTQ2M2MgX2NfbWxfdXNlcnNfOTExNDYzYyBfY19kZl9wcm9qZWN0X21lbWJlcl80MGRmZTU2OCBfY19kZl92aWV3XzZmNTllOWYzMCBfY19kZl9wcm9qZWN0X21lbWJlcl81NzVmODRmNyBfY19kZV91c2Vyc182ZjU5ZTlmMyIsImV4cCI6MTc1MzExNTY2MiwidHlwZSI6InVzZXIiLCJnaXZlbl9uYW1lIjoiT2xpdmVyIiwiaWF0IjoxNzUzMTEyMDYyLCJmYW1pbHlfbmFtZSI6IlphcmF0ZSIsImVtYWlsIjoib3phcmF0ZUBjbG91ZGVyYS5jb20ifQ.nNhXPlaPN7mwe5BEke0QxKKlPUHeTmWFI52SfnIhAASdaT6c4uqC1fPI6NZabF7ZVRD9ayTfbzcD-mUCuWFmXuDbGfmgnV03dojMhyL0m7IluVVDL6y3kSBtp90MBCHatZusoCoYiKNX7pgZYwDppUjcYN8QWynpW1vo7n6XWdI1juRQLBIyMyL4nYh-2Pl7M4PcTrAHgJ8xjal07oJRTRYlvCwIzHKT3RJTuk5RLh1NVydy8ER3I4bL1VafqsctNoh3hfcs5XGUXIp0N5_0vK14jL_hMbaWq7LiKDFh4OuhY8YX10f7GPnoWZQ1KvmUDk4ZMWVZ0NJLz-HVOpugxw'

In [ ]:
client = OpenAI(
	base_url=base_url,
	api_key=OPENAI_API_KEY,
)

## Basic Model Interaction

This section demonstrates the simplest way to interact with our deployed model through the OpenAI package. We'll:
1. Create a client with our model's endpoint and authentication
2. Send a simple message to test the connection
3. Display the model's streaming response

This represents the most straightforward way to interact with the model, similar to how you might use OpenAI's API. The key difference is that we're using our own deployed model through Cloudera AI's infrastructure.

Note: We're using streaming=True in our completion request, which means we'll see the response being generated token by token, providing a more interactive experience.

In [ ]:
message = "Write a one-sentence definition of GenAI."

In [ ]:
completion = client.chat.completions.create(
  model=model_name,
  messages=[{"role":"user","content":message}],
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024,
  stream=True
)

for chunk in completion:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="")

## Enhanced Chat Client Implementation

This section implements a stateful chat client that maintains conversation history and can handle streaming responses. It demonstrates how to build more complex applications while maintaining the simple interface of the basic client.

Key features:
- Conversation history tracking
- Streaming response support
- Configurable parameters
- Error handling

In [ ]:
class ChatClient:
    def __init__(self, model_name: str, base_url: str, api_key):
        self.model_name = model_name
        
        # Set up HTTP client
        if "CUSTOM_CA_STORE" not in os.environ:
            http_client = httpx.Client()
        else:
            http_client = httpx.Client(verify=os.environ["CUSTOM_CA_STORE"])
        
        # Initialize OpenAI client
        self.client = OpenAI(
            base_url=base_url,
            api_key=api_key,
            http_client=http_client,
        )
        
        self.conversation_history: List[Dict[str, str]] = []

    def chat(self, message: str, stream: bool = True) -> str:
        """
        Send a message to the chat model and get the response.
        """
        # Add user message to history
        self.conversation_history.append({"role": "user", "content": message})
        
        try:
            if stream:
                partial_message = ""
                response = self.client.chat.completions.create(
                    model=self.model_name,
                    messages=self.conversation_history + [{"role": "system", "content": "After your thinking, always provide a clear, structured answer."}],
                    temperature=0.6,
                    top_p=0.7,
                    max_tokens=1024,
                    stream=True,
                )
                
                for chunk in response:
                    if chunk.choices[0].delta.content is not None:
                        content = chunk.choices[0].delta.content
                        print(content, end="", flush=True)  # ADD THIS LINE
                        partial_message += content
                
                print()  # ADD THIS LINE for a newline at the end
                final_message = partial_message
            else:
                response = self.client.chat.completions.create(
                    model=self.model_name,
                    messages=self.conversation_history,
                    temperature=0.6,
                    top_p=0.7,
                    max_tokens=512,
                    stream=False,
                )
                
                complete_response = response.choices[0].message.content
                final_message = complete_response
                
            if final_message:
                self.conversation_history.append({"role": "assistant", "content": final_message})
            
            return final_message
            
        except Exception as e:
            print(f"Error in chat method: {str(e)}")
            raise
    
    def get_history(self) -> List[Dict[str, str]]:
        """Get the conversation history."""
        return self.conversation_history
    
    def clear_history(self):
        """Clear the conversation history."""
        self.conversation_history = []

In [ ]:
# Initialize the chat client
chat_client = ChatClient(model_name, base_url,OPENAI_API_KEY)

In [ ]:
message = """in 6 sentences or less explain how weights get update during model training process of a neural network. Explain this to a 6 year old'"""

In [ ]:
# For streaming responses (will print as it receives chunks):
response = chat_client.chat(message, stream=True)

In [ ]:
message2 = "now do the same thing but explain learning rate. 5 sentences or less"

In [ ]:
# For streaming responses (will print as it receives chunks):
response = chat_client.chat(message2, stream=True)

## Model Switching Demonstration

One of the key benefits of Cloudera AI is the ability to easily switch between different models. Here we'll demonstrate this by changing to the second model while using the same code structure.

For this section, we'll use our second model's information:
- URL goes into `llama_70b_base_url` (remember to clip after /v1)
- Model ID goes into `llama_70b_model_name`

In [ ]:
# Initialize the chat client
new_chat_client = ChatClient(llama_model_name,llama_url,OPENAI_API_KEY)

In [ ]:
message3 = "in 5 sentences or less what is a learning rate in neural networks?"
response_llama_33 = new_chat_client.chat(message3, stream=True)
#print(response_ds)